In [129]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [130]:
custom_stopwords = ['echo', 'alexa', 'dot', "star", 'amazon', 'prime', '2nd', 'generation', "fire", "stick", "firestick", "skype", "facetime", '1st', '3rd', '4th', '5th', "hub", "hulu", 'google', 'netflix', 'youtube', 'philip', 'tp-link', 'fourth', 'roku', "siri", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "...", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def negation_handler(sentence):	

    """Handle negations using WordNet. See https://github.com/UtkarshRedd/Negation_handling for a clear explenation."""

    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't", "no", "without"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [176]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 3-4 stelle
dataset.tail()

,rating,date,variation,verified_reviews,feedback
3144,5,30-Jul-18,Black Dot,love it,1
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [177]:
#  CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


# dataset = create_balanced_dataset(dataset)

In [178]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v).
    This was done in order to have compatibility with the wordnet lemmatizer.
    For example the pos "JJ" is transformed in "a".
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza.
    Ritorna sia il la lista tokenizzata ma come stringa sia come lista di tokens, dunque ritorna due elementi"""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    detokenizer = TreebankWordDetokenizer()

    tokenized_reviews = []
    sent_tokenized_reviews = []
    pos_reviews = []
    for review in list_of_text: #pulisce le recensioni
        review = re.sub(r'\d+', '', review) # elimina i numeri
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = negation_handler(tokens)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [(w, pos) for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords]
        sent_tokenized_reviews.append([w for w, pos in clean_tokens])
        tokenized_reviews.append(detokenizer.detokenize([w for w, pos in clean_tokens]))
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [179]:
new_text, new_sent_tok = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, False, pos_list)
# contengono una lista di tutte le frasi pre processate, nella prima variabile in stringa, nella seconda in tokens 
bigrams = Phrases(new_sent_tok) #estrare le collocazioni tramite PMI
bigrams[new_sent_tok][12]

total number of tokens extracted are: 2766


['expect', '....']

In [180]:
# Elimino token con freq minore di 5
tot_tokens = []

for sent in [x for x in bigrams[new_sent_tok]]:
    for tok in sent:
        tot_tokens.append(tok)

freqs = nltk.FreqDist(tot_tokens)
cleaned_reviews = []

for sent in [x for x in bigrams[new_sent_tok]]:
    clean_sent = []
    for tok in sent:
        if freqs[tok] > 4:
            clean_sent.append(tok)
    cleaned_reviews.append(clean_sent)

# qui invece creo una nuova lista che contiene tutti i token e la relativa pos nella forma "good$JJ"
pos_reviews = [["$".join(x) for x in pos_tag(rev)] for rev in cleaned_reviews]
pos_reviews[10]

['purchase$NN',
 'mother$NN',
 'problem$NN',
 'give$VBP',
 'something$NN',
 'try$NN',
 'come$VBP',
 'leave$VBP',
 'around$IN',
 'fast$RB',
 'like$IN',
 'enjoy$NN',
 'little$RB',
 'big$JJ',
 'thing$NN',
 'play$NN',
 'song$NN',
 'time$NN',
 'cook$NN']

In [181]:
# Questa parte estrare le parole più rilevanti per ogni categoria, ma non la uso alla fine.
# Lo riprenderò più avanti

neg_tokenized = [rev for rev, feedback in zip(cleaned_reviews, dataset["feedback"]) if feedback == 0]
pos_tokenized = [rev for rev, feedback in zip(cleaned_reviews, dataset["feedback"]) if feedback == 1]
neg_tokenized = [w for rev in neg_tokenized for w in rev]
pos_tokenized = [w for rev in pos_tokenized for w in rev]

import math 

def get_pos_neg_score(pos, neg, threshold = 0.30):
    total_tokens = pos + neg
    total_tokens_freq = nltk.FreqDist(total_tokens)
    pos_freq = nltk.FreqDist(pos)
    neg_freq = nltk.FreqDist(neg)
    data_dict = {"pos": {}, "neg": {}}

    for w in pos:
        p = total_tokens_freq[w] / len(total_tokens)
        p_category = pos_freq[w] / total_tokens_freq[w]
        h = -math.log2(p)
        score = p_category * h * p
        data_dict["pos"][w] = score
    
    for w in neg:
        p = total_tokens_freq[w] / len(total_tokens)
        p_category = neg_freq[w] / total_tokens_freq[w]
        h = -math.log2(p)
        score = p_category * h * p
        data_dict["neg"][w] = score

    risultato = {}

    for label in ["pos", "neg"]:
        values_list = list(data_dict[label].values())
        min_values = min(values_list)
        max_values = max(values_list)
        for k in data_dict[label]:
            val = data_dict[label][k]
            data_dict[label][k] = (val- min_values)/(max_values-min_values)

    for w in total_tokens:

        try:
            pos_score = data_dict["pos"][w]
        except KeyError:
            # risultato[w] = 1
            pass

        try:
            neg_score = data_dict["neg"][w]
        except KeyError:
            # risultato[w] = 1
            pass


        risultato[w] = max([pos_score, neg_score])

    # return data_dict
    return {k: v for k, v in sorted(risultato.items(), key = lambda item: item[1], reverse=True)}
    
data_dict = get_pos_neg_score(pos_tokenized, neg_tokenized, threshold=0)
data_dict

{'love': 1.0,
 'not': 1.0,
 'tech_savvy': 1.0,
 'device': 0.8648437904399189,
 'include': 0.8648437904399189,
 'grandkids': 0.8648437904399189,
 'cut': 0.8648437904399189,
 'place': 0.7240030054062944,
 'would': 0.7240030054062944,
 'work': 0.6993366681294355,
 'great': 0.6977542072262416,
 'get': 0.6747608154592976,
 'dad': 0.6747608154592976,
 'father_day': 0.6747608154592976,
 'classroom': 0.6747608154592976,
 'comfortable': 0.6747608154592976,
 'sooner': 0.6747608154592976,
 'time': 0.612980640878305,
 'sit': 0.612980640878305,
 'course': 0.612980640878305,
 'try': 0.5938337210336337,
 'use': 0.5852081560151066,
 'malfunction': 0.5852081560151066,
 'buy': 0.5610881162499615,
 'husband': 0.5610881162499615,
 'saw': 0.5610881162499615,
 'thing': 0.49041767976809375,
 'base': 0.49041767976809375,
 'one': 0.4847468460507719,
 'mother': 0.4847468460507719,
 'daughter': 0.4847468460507719,
 'sale': 0.4847468460507719,
 'product': 0.48097443546119756,
 'versatile': 0.48097443546119756,
 '

In [182]:
w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007)

In [183]:
w2v_model.train(cleaned_reviews, total_examples=len(cleaned_reviews), epochs=100, report_delay=1)

(1663348, 2193800)

In [184]:
w2v_model.wv.most_similar("great", topn = 10)

[('work', 0.2645948827266693),
 ('satisfied', 0.19260777533054352),
 ('love', 0.18071243166923523),
 ('charm', 0.18004941940307617),
 ('flawlessly', 0.1788301318883896),
 ('many_thing', 0.17521290481090546),
 ('course', 0.17247360944747925),
 ('shipping', 0.16908293962478638),
 ('excellent', 0.1676042377948761),
 ('kitchen', 0.16568666696548462)]

In [185]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('package', -0.033238790929317474),
 ('suppose', -0.03372375667095184),
 ('people', -0.03378671035170555),
 ('never', -0.03396548703312874),
 ('sonos', -0.0390961617231369),
 ('annoy', -0.0394415408372879),
 ('close', -0.04699898511171341),
 ('default', -0.050131235271692276),
 ('expensive', -0.06306134909391403),
 ('app', -0.06488937884569168)]

In [186]:
# tfidf weighting sui token, per ora senza pos. Mi sono reso conto che la pos per averla più accurata la devo fare prima del pre processing.
 
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(cleaned_reviews)
review = 15
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

benefit 0.32633520817571116
come 0.23328841732724445
couple 0.29703902023569817
day 0.18880233480487343
different 0.2607284750446854
household 0.3008270617286897
learn 0.222350996368215
like 0.1463344158156626
lot_fun 0.2902667364243624
member 0.3200158010669249
new 0.3761371904696644
toy 0.3094554757625976
try 0.21104537606605894
use 0.13200924198830968


In [187]:
df = pd.DataFrame(tfidf_model.toarray(), columns = tfidf.get_feature_names_out())
df

,'ll,'re,'ve,....,.....,......,ability,able,absolutely,absolutely_love,...,worth_every,would,would_recommend,wrong,yard,year,year_old,yell,yes,yet
0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2539,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2540,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2541,0.0,0.000000,0.0,0.154716,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [188]:
g_model = []
# google_model = api.load('word2vec-google-news-300', True)
# g_model = KeyedVectors.load_word2vec_format(google_model, binary=True)

In [230]:
def review_vectors(tokens, size, weights, pretrained = False):

    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * perso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0
    if len(tokens) != len(weights):
        print("nope")
    for word, weight in zip(tokens, weights):
        word = word.split("$")[0]
        try:
            if pretrained:
                vec += g_model[word] * weight
            else:
                vec += w2v_model.wv[word] * weight
            count +=1
        except KeyError:
            print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [231]:
# qui viene creato per ogni recensione il vettore

w2v_reviews = np.zeros((len(cleaned_reviews), 300))
for i in range(len(cleaned_reviews)):
    w2v_reviews[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], 300, tfidf_model[i,:].data, False)
w2v_df = pd.DataFrame(w2v_reviews)
w2v_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.018835,0.114063,-0.005872,-0.037132,-0.019159,-0.106562,-0.032998,0.036485,0.011260,-0.024760,...,0.052651,-0.010966,0.016686,-0.031776,0.020766,-0.008479,-0.043000,0.084970,-0.016822,0.053334
1,0.018835,0.114063,-0.005872,-0.037132,-0.019159,-0.106562,-0.032998,0.036485,0.011260,-0.024760,...,0.052651,-0.010966,0.016686,-0.031776,0.020766,-0.008479,-0.043000,0.084970,-0.016822,0.053334
2,0.054088,0.054925,0.004768,0.037074,-0.012668,-0.027078,0.103468,0.133006,0.012010,-0.037656,...,-0.081107,0.027491,0.026664,-0.054770,0.000420,0.044232,-0.086418,-0.032396,-0.003430,-0.046461
3,-0.031830,0.004683,-0.031511,0.035185,0.090978,-0.045289,0.094294,0.039568,-0.043606,0.078406,...,-0.046349,-0.012908,0.019170,0.062864,0.028851,0.003115,0.039124,0.033055,0.046597,-0.088817
4,0.001675,-0.025436,-0.033233,-0.007951,-0.019409,-0.016291,0.054255,0.116747,-0.022510,-0.128829,...,-0.009673,-0.021017,0.044019,0.003073,0.103746,0.050996,0.001409,0.017452,0.003311,-0.023255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,0.018835,0.114063,-0.005872,-0.037132,-0.019159,-0.106562,-0.032998,0.036485,0.011260,-0.024760,...,0.052651,-0.010966,0.016686,-0.031776,0.020766,-0.008479,-0.043000,0.084970,-0.016822,0.053334
2539,-0.029426,0.048604,-0.028471,0.100392,0.019874,-0.104132,0.148054,0.107587,-0.082930,-0.008598,...,-0.008962,-0.023171,-0.007217,-0.060348,-0.042541,0.039134,0.023135,-0.069694,0.092551,0.028465
2540,0.044208,0.071327,0.082717,0.037706,-0.015875,0.004365,0.068997,0.115959,0.009944,0.000178,...,-0.029193,0.071147,0.057531,-0.092072,0.021728,0.055076,0.046241,-0.039149,-0.029371,0.028971
2541,-0.009945,0.131182,-0.037812,-0.013819,-0.007487,-0.093861,0.048825,0.173389,0.027627,-0.065191,...,-0.030953,0.030750,0.051448,-0.004972,0.036655,0.138683,-0.024324,0.002024,0.082460,-0.050042


In [240]:
X_train, X_test, Y_train, Y_test = train_test_split(w2v_reviews, dataset["feedback"].values, test_size=0.20, random_state=10)
Y_train

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [241]:
svm = LinearSVC(max_iter=3000)
svm_model = svm.fit(X_train, Y_train)
predictions = svm_model.predict(X_test)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.68      0.57      0.62        44
           1       0.96      0.97      0.97       465

    accuracy                           0.94       509
   macro avg       0.82      0.77      0.79       509
weighted avg       0.94      0.94      0.94       509



In [242]:
# risultati migliori:

# w2v + tf-idf weights considering pos (good$JJ) disambiguation also! + svm and word count > 4

# w2v configuration:
# w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007) and 100 epochs for training

# otherwise the balanced dataset performs better


In [243]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [244]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

In [245]:
grid = GridSearchCV(SVC(), param_grid, refit ='f1_macro', cv=kfold, verbose=3)
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.862 total time=   0.5s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.882 total time=   0.3s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.857 total time=   0.5s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.907 total time=   0.3s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.879 total time=   0.4s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.855 total time=   0.1s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.860 total time=   0.1s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.826 total time=   0.2s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.865 total time=   0.3s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             refit='f1_macro', verbose=3)

In [246]:
print(grid.best_params_)

{'C': 1, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}


In [247]:
grid_predictions = grid.predict(X_test)

    # print classification report
print(classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.71      0.66      0.68        44
           1       0.97      0.97      0.97       465

    accuracy                           0.95       509
   macro avg       0.84      0.82      0.83       509
weighted avg       0.95      0.95      0.95       509

